In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd

dataframe = pd.read_csv('out.csv', header=None, index_col=0)
dataframe.head()

,1,2
0,,
155500,курский губернатор роман старовойт сегодня со ...,24945
23628,в результате обстрела села плехово в курской о...,24945
64741,в результате обстрела села плехово суджанского...,24945
89718,тридцать летний мужчина погиб в результате обс...,24945
66100,мужчина погиб в результате обстрела всу села п...,24945


In [ ]:
from itertools import combinations

dataset_raw = {'first': [], 'second': [], 'similarity': []}

for (first_index, first_text, first_label), (second_index, second_text, second_label) in combinations(dataframe.itertuples(), r=2):
    dataset_raw['first'].append(first_text)
    dataset_raw['second'].append(second_text)
    if first_label == second_label:
        dataset_raw['similarity'].append(1)
    else:
        dataset_raw['similarity'].append(0)

In [ ]:
dataset = pd.DataFrame.from_dict(dataset_raw)
dataset.head()

,first,second,similarity
0,курский губернатор роман старовойт сегодня со ...,в результате обстрела села плехово в курской о...,1
1,курский губернатор роман старовойт сегодня со ...,в результате обстрела села плехово суджанского...,1
2,курский губернатор роман старовойт сегодня со ...,тридцать летний мужчина погиб в результате обс...,1
3,курский губернатор роман старовойт сегодня со ...,мужчина погиб в результате обстрела всу села п...,1
4,курский губернатор роман старовойт сегодня со ...,один человек погиб в результате обстрела села ...,1


In [ ]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device=device)

In [ ]:
from sklearn.utils import shuffle

dataset_reduced = shuffle(dataset)[:len(dataset) // 8]

In [ ]:
input_examples = []

for index, first_text, second_text, similarity in dataset_reduced.itertuples():
    input_examples.append(InputExample(texts=[first_text, second_text], label=float(similarity)))

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(input_examples, test_size=0.2, shuffle=False)
val_dataset, test_dataset = train_test_split(val_dataset, test_size=0.5, shuffle=False)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=32)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(val_dataset)

In [ ]:
import math

num_epochs = 10

warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path='model_output',
          show_progress_bar=True)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1845 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_dataset, name='test')

In [ ]:
test_evaluator(model, output_path='model_output')

0.2249351438291963

In [ ]:
!zip -r /content/sentence_transformers.zip /content/model_output

  adding: content/model_output/ (stored 0%)
  adding: content/model_output/tokenizer_config.json (deflated 75%)
  adding: content/model_output/special_tokens_map.json (deflated 42%)
  adding: content/model_output/sentence_bert_config.json (deflated 4%)
  adding: content/model_output/similarity_evaluation_test_results.csv (deflated 45%)
  adding: content/model_output/vocab.txt (deflated 45%)
  adding: content/model_output/config.json (deflated 45%)
  adding: content/model_output/model.safetensors (deflated 7%)
  adding: content/model_output/eval/ (stored 0%)
  adding: content/model_output/eval/similarity_evaluation_results.csv (deflated 55%)
  adding: content/model_output/2_Dense/ (stored 0%)
  adding: content/model_output/2_Dense/config.json (deflated 22%)
  adding: content/model_output/2_Dense/pytorch_model.bin (deflated 7%)
  adding: content/model_output/tokenizer.json (deflated 67%)
  adding: content/model_output/modules.json (deflated 62%)
  adding: content/model_output/1_Pooling/ 

In [ ]:
from google.colab import files
files.download("/content/sentence_transformers.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

shutil.copyfile('/content/sentence_transformers.zip', '/content/drive/MyDrive/sentence_transformers.zip')

'/content/drive/MyDrive/sentence_transformers.zip'